In [3]:
! git clone https://github.com/OctoberChang/klcpd_code.git

fatal: destination path 'klcpd_code' already exists and is not an empty directory.


In [4]:
!pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import numpy as np
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, TensorDataset

import pytorch_lightning as pl
from typing import List, Tuple
from sklearn.metrics.pairwise import euclidean_distances

import random

#### Utils

In [6]:
from __future__ import print_function
import os
import numpy as np
import scipy.io as sio
import sklearn.metrics
import _pickle as pickle


def load_data(data_path):
    data = sio.loadmat(data_path)
    return data['Y'], data['L']

def load_matlab_v1_log(data_path):
    eval_log = sio.loadmat(data_path)
    ret_dict = {'Y_tst': eval_log['Y_tst'],
                'L_tst': eval_log['L_tst'],
                'Y_tst_pred': eval_log['Y_tst_pred'],
                'L_tst_pred': eval_log['err'][:, 1], # use MSE as predict score
                'err': eval_log['err']}
    return ret_dict

def load_matlab_v2_log(data_path):
    eval_log = sio.loadmat(data_path)
    ret_dict = {'Y_tst': eval_log['Y_tst'],
                'L_tst': eval_log['L_tst'],
                'Y_tst_pred': None,
                'L_tst_pred': eval_log['Y_tst_pred'],
                'err': None}
    return ret_dict

def load_python_log(data_path):
    eval_log = pickle.load(open(data_path, 'rb'))
    ret_dict = {'Y_tst': eval_log['Y_true'],
                'L_tst': eval_log['L_true'],
                'Y_tst_pred': eval_log['Y_pred'],
                'L_tst_pred': eval_log['L_pred'],
                'err': None}
    return ret_dict

def compute_auc(eval_dict):
    L_true = eval_dict['L_tst'].flatten()
    L_pred = eval_dict['L_tst_pred'].flatten()
    # print('L_true', L_true.shape, 'L_pred', L_pred.shape)
    fp_list, tp_list, thresholds = sklearn.metrics.roc_curve(L_true, L_pred)
    auc = sklearn.metrics.auc(fp_list, tp_list)
    return fp_list, tp_list, auc

def compute_average_roc(tprs, base_fpr):
    tprs = np.array(tprs)
    mean_tprs = tprs.mean(axis=0)
    std = tprs.std(axis=0)

    tprs_upper = np.minimum(mean_tprs + std, 1)
    tprs_lower = mean_tprs - std
    return mean_tprs

def forecast_loss(eval_dict):
    assert(eval_dict['Y_tst_pred'] is not None)
    sqr_err = np.sum((eval_dict['Y_tst'] - eval_dict['Y_tst_pred'])**2, axis=1)
    abs_err = np.sum(abs(eval_dict['Y_tst'] - eval_dict['Y_tst_pred']), axis=1)
    mse_mean = np.mean(sqr_err)
    mae_mean = np.mean(abs_err)
    return mse_mean, mae_mean

def print_auc_table(result_array, all_methods):
    # print auc for latex table
    print('metric', end='')
    for i, method in enumerate(all_methods):
        print(' & %s' % (method), end='')
    print('')
    print('AUC', end='')
    for i, method in enumerate(all_methods):
        print(' & %.4f' % (np.mean(result_array[i, :])), end='')
    print('')

#### Data

In [7]:
import os
import numpy as np
import scipy.io as sio
import math
import torch
from torch.autograd import Variable

In [8]:
class DataLoader(object):
    def __init__(self, args, trn_ratio=0.6, val_ratio=0.8):
        self.cuda = args.cuda
        self.data_path = args.data_path
        self.p_wnd_dim = 25
        self.f_wnd_dim = args.wnd_dim
        self.sub_dim = args.sub_dim
        self.batch_size = args.batch_size

        # load data
        self.load_data(trn_ratio=trn_ratio, val_ratio=val_ratio)

        # prepare data
        self.prepare_data()

        # split data into trn/val/tst set
        self.split_data()

    # load data
    def load_data(self, trn_ratio=0.6, val_ratio=0.8):
        assert(os.path.lexists(self.data_path))
        dataset = sio.loadmat(self.data_path)
        self.Y = dataset['Y']                                   # Y: time series data, time length x number of variables
        self.L = dataset['L']                                   # L: label of anomaly, time length x 1
        self.T, self.D = self.Y.shape                           # T: time length; D: variable dimension
        self.n_trn = int(np.ceil(self.T * trn_ratio))           # n_trn: first index of val set
        self.n_val = int(np.ceil(self.T * val_ratio))           # n_val: first index of tst set
        self.var_dim = self.D * self.sub_dim

    # prepare subspace data (Hankel matrix)
    def prepare_data(self):
        # T x D x sub_dim
        self.Y_subspace = np.zeros((self.T, self.D, self.sub_dim))
        for t in range(self.sub_dim, self.T):
            for d in range(self.D):
                self.Y_subspace[t, d, :] = self.Y[t-self.sub_dim+1:t+1, d].flatten()

        # Y_subspace is now T x (Dxsub_dim)
        self.Y_subspace = self.Y_subspace.reshape(self.T, -1)

    # split data into trn/val/tst set
    def split_data(self):
        trn_set_idx = range(self.p_wnd_dim, self.n_trn)
        val_set_idx = range(self.n_trn, self.n_val)
        tst_set_idx = range(self.n_val, self.T)
        print('n_trn ', len(trn_set_idx), 'n_val ', len(val_set_idx), 'n_tst ', len(tst_set_idx))
        self.trn_set = self.__batchify(trn_set_idx)
        self.val_set = self.__batchify(val_set_idx)
        self.tst_set = self.__batchify(tst_set_idx)

    # convert augmented data in Hankel matrix to origin time series
    # input: X_f, whose shape is batch_size x seq_len x (D*sub_dim)
    # output: Y_t, whose shape is batch_size x D
    def repack_data(self, X_f, batch_size):
        Y_t = X_f[:, 0, :].contiguous().view(batch_size, self.D, self.sub_dim)
        return Y_t[:, :, -1]

    def __batchify(self, idx_set):
        n = len(idx_set)
        L = torch.zeros((n, 1))                             # anomaly label
        Y = torch.zeros((n, self.D))                        # true signal
        X_p = torch.zeros((n, self.p_wnd_dim, self.var_dim))  # past window buffer
        X_f = torch.zeros((n, self.f_wnd_dim, self.var_dim))  # future window buffer

        # XXX: dirty trick to augment the last buffer
        data = np.concatenate((self.Y_subspace, self.Y_subspace[-self.f_wnd_dim:, :]))
        for i in range(n):
            l = idx_set[i] - self.p_wnd_dim
            m = idx_set[i]
            u = idx_set[i] + self.f_wnd_dim
            X_p[i, :, :] = torch.from_numpy(data[l:m, :])
            X_f[i, :, :] = torch.from_numpy(data[m:u, :])
            Y[i, :] = torch.from_numpy(self.Y[m, :])
            L[i] = torch.from_numpy(self.L[m])
        return {'X_p': X_p, 'X_f': X_f, 'Y': Y, 'L': L}

    def get_batches(self, data_set, batch_size, shuffle=False):
        X_p, X_f = data_set['X_p'], data_set['X_f']
        Y, L = data_set['Y'], data_set['L']
        length = len(Y)
        if shuffle:
            index = torch.randperm(length)
        else:
            index = torch.LongTensor(range(length))
        s_idx = 0
        while (s_idx < length):
            e_idx = min(length, s_idx + batch_size)
            excerpt = index[s_idx:e_idx]
            X_p_batch, X_f_batch = X_p[excerpt], X_f[excerpt]
            Y_batch, L_batch = Y[excerpt], L[excerpt]
            if self.cuda:
                X_p_batch = X_p_batch.cuda()
                X_f_batch = X_f_batch.cuda()
                Y_batch = Y_batch.cuda()
                L_batch = L_batch.cuda()

            data = [Variable(X_p_batch),
                    Variable(X_f_batch),
                    Variable(Y_batch),
                    Variable(L_batch)]
            yield data
            s_idx += batch_size

#### Setting pipeline parameters

In [9]:
import argparse
import os
import glob

import sys
import os

py_file_location = "/content/klcpd_code"  
sys.path.append(os.path.abspath(py_file_location))

import mmd_util

In [66]:
###$ python run_klcpd.py --dataroot ./data --dataset beedance --wnd_dim_list 25 --max_iter 2000 --batch_size 64 
# dataset
parser = argparse.ArgumentParser(description='interface of running experiments for MMD baselines')
#parser.add_argument('--dataroot', type=str, default='beedance', required=True, help='[ ./data/simulation | ./data ] prefix path to data directory')
parser.add_argument('--dataroot', type=str, default='/content/klcpd_code/data', help = 'default version')
parser.add_argument('--dataset', type=str, default='beedance', help='dataset name ')
parser.add_argument('--gpu', type=int, default=0, help='gpu device id')

# hyperparameters for grid search
parser.add_argument('--wnd_dim_list', type=int, nargs='+', default=[5, 10, 15, 20, 25, 30], help='list of window dimensions')
parser.add_argument('--lambda_ae', type=float, default=1e-3, help='list of lambda_ae, coefficient of reconstruction loss')
parser.add_argument('--lambda_real', type=float, default=1, help='list of lambda_real, coefficient of MMD2(X_p_enc, X_f_enc)')
parser.add_argument('--max_iter', type=int, default=2000, help='max iteration for training')
parser.add_argument('--batch_size', type=int, default=64, help='batch_size for training')
parser.add_argument('--eval_freq', type=int, default=25, help='evaluation frequency per batch updates')
parser.add_argument('--weight_clip', type=float, default=.1, help='weight clipping for netD')

# experiment log
parser.add_argument('--save_dir', type=str, default='experiment_log', help='experiment directory for saving train log and models')

# sanity check
args, unknown = parser.parse_known_args()

dataroot = args.dataroot
dataset = args.dataset
gpu = args.gpu
wnd_dim_list = args.wnd_dim_list
lambda_ae = args.lambda_ae
lambda_real = args.lambda_real
max_iter = args.max_iter
batch_size = args.batch_size
eval_freq = args.eval_freq
weight_clip = args.weight_clip

In [67]:
dataroot, dataset, gpu, wnd_dim_list, lambda_ae, lambda_real, max_iter, batch_size, eval_freq, weight_clip

('/content/klcpd_code/data',
 'hasc',
 0,
 [5, 10, 15, 20, 25, 30],
 0.001,
 1,
 2000,
 64,
 25,
 0.1)

In [69]:
    for wnd_dim in wnd_dim_list:
        data_dir = os.path.join(dataroot, dataset)
        for data_path in glob.glob('%s/*.mat' % (data_dir)):
            data_name = data_path.split('/')[-1].split('.')[0]
            save_name = '%s.wnd-%d.lambda_ae-%f.lambda_real-%f.clip-%f' % (data_name, wnd_dim, lambda_ae, lambda_real, weight_clip)
            save_path = '%s/%s' % (args.save_dir, save_name)
            trn_log_path = '%s.trn.log' % (save_path)
            option = '--data_path %s --wnd_dim %d --lambda_ae %f --lambda_real %f --weight_clip %f --max_iter %d --batch_size %d --eval_freq %d --save_path %s' \
                % (data_path, wnd_dim, lambda_ae, lambda_real, weight_clip, max_iter, batch_size, eval_freq, save_path)
            cmd = '! CUDA_VISIBLE_DEVICES=%d python -u /content/klcpd_code/klcpd.py %s 2>&1 | tee %s' % (gpu, option, trn_log_path)
            #print(cmd)
            #os.system(cmd)

#### OS command

In [70]:
cmd

'! CUDA_VISIBLE_DEVICES=0 python -u /content/klcpd_code/klcpd.py --data_path /content/klcpd_code/data/hasc/hasc-1.mat --wnd_dim 30 --lambda_ae 0.001000 --lambda_real 1.000000 --weight_clip 0.100000 --max_iter 2000 --batch_size 64 --eval_freq 25 --save_path experiment_log/hasc-1.wnd-30.lambda_ae-0.001000.lambda_real-1.000000.clip-0.100000 2>&1 | tee experiment_log/hasc-1.wnd-30.lambda_ae-0.001000.lambda_real-1.000000.clip-0.100000.trn.log'

In [31]:
save_path

'experiment_log/beedance-2.wnd-30.lambda_ae-0.001000.lambda_real-1.000000.clip-0.100000'

In [32]:
option

'--data_path /content/klcpd_code/data/beedance/beedance-2.mat --wnd_dim 30 --lambda_ae 0.001000 --lambda_real 1.000000 --weight_clip 0.100000 --max_iter 2000 --batch_size 64 --eval_freq 25 --save_path experiment_log/beedance-2.wnd-30.lambda_ae-0.001000.lambda_real-1.000000.clip-0.100000'

In [42]:
os.makedirs(save_path)

In [43]:
isExist = os.path.exists(save_path)
isExist

True

In [35]:
cmd

'! CUDA_VISIBLE_DEVICES=0 python -u /content/klcpd_code/klcpd.py --data_path /content/klcpd_code/data/beedance/beedance-2.mat --wnd_dim 30 --lambda_ae 0.001000 --lambda_real 1.000000 --weight_clip 0.100000 --max_iter 2000 --batch_size 64 --eval_freq 25 --save_path experiment_log/beedance-2.wnd-30.lambda_ae-0.001000.lambda_real-1.000000.clip-0.100000 2>&1 | tee experiment_log/beedance-2.wnd-30.lambda_ae-0.001000.lambda_real-1.000000.clip-0.100000.trn.log'

In [45]:
!rm /content/experiment_log/*

rm: cannot remove '/content/experiment_log/beedance-2.wnd-30.lambda_ae-0.001000.lambda_real-1.000000.clip-0.100000': Is a directory


#### Experiment with Beedance Dataset

In [46]:
! CUDA_VISIBLE_DEVICES=0 python -u /content/klcpd_code/klcpd.py --data_path /content/klcpd_code/data/beedance/beedance-2.mat --wnd_dim 30 --lambda_ae 0.001000 --lambda_real 1.000000 --weight_clip 0.100000 --max_iter 2000 --batch_size 64 --eval_freq 25 --save_path experiment_log/beedance-2.wnd-30.lambda_ae-0.001000.lambda_real-1.000000.clip-0.100000 2>&1 | tee experiment_log/beedance-2.wnd-30.lambda_ae-0.001000.lambda_real-1.000000.clip-0.100000.trn.log

Namespace(CRITIC_ITERS=5, RNN_hid_dim=10, batch_size=64, cuda=True, data_path='/content/klcpd_code/data/beedance/beedance-2.mat', eval_freq=25, gpu=0, grad_clip=10.0, lambda_ae=0.001, lambda_real=1.0, lr=0.0003, max_iter=2000, momentum=0.0, optim='adam', random_seed=1126, save_path='experiment_log/beedance-2.wnd-30.lambda_ae-0.001000.lambda_real-1.000000.clip-0.100000', sub_dim=1, trn_ratio=0.6, val_ratio=0.8, weight_clip=0.1, weight_decay=0.0, wnd_dim=30)
Using GPU device 0
n_trn  650 n_val  225 n_tst  224
NetG(
  (rnn_enc_layer): GRU(3, 10, batch_first=True)
  (rnn_dec_layer): GRU(3, 10, batch_first=True)
  (fc_layer): Linear(in_features=10, out_features=3, bias=True)
)
NetD(
  (rnn_enc_layer): GRU(3, 10, batch_first=True)
  (rnn_dec_layer): GRU(10, 3, batch_first=True)
)
netG has number of parameters: 933
netD has number of parameters: 585
sigma_list: tensor([0.0476, 0.0952, 0.1904, 0.3808, 0.7615], device='cuda:0')
n_batchs 11 batch_size 64
start training: lambda_ae 0.001 lambda_re

It can be seen that test metrics improved significantly. Even if we relaxed stopping criteria.

In [28]:
! CUDA_VISIBLE_DEVICES=0 python -u klcpd.py --data_path /content/klcpd_code/data/beedance/beedance-2.mat --wnd_dim 30 --lambda_ae 0.001000 --lambda_real 1.000000 --weight_clip 0.100000 --max_iter 2000 --batch_size 64 --eval_freq 25 --save_path experiment_log/beedance-2.wnd-30.lambda_ae-0.001000.lambda_real-1.000000.clip-0.100000 2>&1 | tee experiment_log/beedance-2.wnd-30.lambda_ae-0.001000.lambda_real-1.000000.clip-0.100000.trn.log

python3: can't open file 'klcpd.py': [Errno 2] No such file or directory


#### Saving file with results

In [61]:
file = open('/content/experiment_log/beedance-2.wnd-30.lambda_ae-0.001000.lambda_real-1.000000.clip-0.100000.trn.log', 'r')


lines = file.read().splitlines()

lines[-9][-60:]

'[best_val_auc 0.640271 best_tst_auc 0.648864 best_epoch 100]'

Results after 100s run

In [ ]:
!python -u klcpd.py --data_path /content/data/beedance/beedance-2.mat --wnd_dim 30 --lambda_ae 0.001000 --lambda_real 1.000000 --weight_clip 0.100000 --max_iter 2000 --batch_size 64 --eval_freq 25 --save_path experiment_log/beedance-2.wnd-30.lambda_ae-0.001000.lambda_real-1.000000.clip-0.100000 2>&1 | tee experiment_log/beedance -2 .wnd-30.lambda_ae-0.001000.lambda_real-1.000000.clip-0.100000.trn.log

tee: invalid option -- '2'
Try 'tee --help' for more information.


More prolonged run

In [ ]:
! python run_klcpd.py --dataroot /content/data --dataset beedance --wnd_dim_list 25 --max_iter 2000 --batch_size 64

Streaming output truncated to the last 5000 lines.
[  268/ 2000] [    6/    6] [   268] D_mmd2 1.4732e+00 G_mmd2 1.5271e+00 mmd2_real 8.5491e-01 real_L2 0.048817 fake_L2 0.014548
[  269/ 2000] [    6/    6] [   269] D_mmd2 1.5445e+00 G_mmd2 1.3058e+00 mmd2_real 7.7069e-01 real_L2 0.048259 fake_L2 0.015569
[  270/ 2000] [    6/    6] [   270] D_mmd2 1.3711e+00 G_mmd2 1.4277e+00 mmd2_real 7.8935e-01 real_L2 0.046314 fake_L2 0.012369
[  271/ 2000] [    6/    6] [   271] D_mmd2 1.3139e+00 G_mmd2 1.6327e+00 mmd2_real 7.7862e-01 real_L2 0.046480 fake_L2 0.013108
[  272/ 2000] [    6/    6] [   272] D_mmd2 1.2692e+00 G_mmd2 1.3267e+00 mmd2_real 5.6642e-01 real_L2 0.043844 fake_L2 0.012070
[  273/ 2000] [    6/    6] [   273] D_mmd2 1.3144e+00 G_mmd2 1.2249e+00 mmd2_real 7.2323e-01 real_L2 0.046562 fake_L2 0.014668
[  274/ 2000] [    6/    6] [   274] D_mmd2 1.3430e+00 G_mmd2 1.0528e+00 mmd2_real 6.9688e-01 real_L2 0.045552 fake_L2 0.014345
iter  274 tm 0.72m val_mse -1.0 val_mae -1.0 val_auc 

Results are better, MMD_loss has e-2 order

#### Saving log files and .py scripts

We obtain logs of train procedure and we saved them and all .py sripts which we used. We modified slightly architecture of the authors

In [98]:
!mkdir 2zip
#!find /content/klcpd_code '*.py' -exec cp -vuni '{}' "../content/2zip" ";"
!find /content/klcpd_code -name '*.py' -exec cp -prv '{}' '../content/2zip' ';'
!cp -R /content/experiment_log ../content/2zip


'/content/klcpd_code/optim.py' -> '../content/2zip/optim.py'
'/content/klcpd_code/median_heuristic.py' -> '../content/2zip/median_heuristic.py'
'/content/klcpd_code/klcpd.py' -> '../content/2zip/klcpd.py'
'/content/klcpd_code/util.py' -> '../content/2zip/util.py'
'/content/klcpd_code/mmd_util.py' -> '../content/2zip/mmd_util.py'
'/content/klcpd_code/run_klcpd.py' -> '../content/2zip/run_klcpd.py'
'/content/klcpd_code/data_loader.py' -> '../content/2zip/data_loader.py'


In [99]:
!zip -r /content/file.zip /content/2zip

updating: content/2zip/ (stored 0%)
updating: content/2zip/optim.py (deflated 68%)
updating: content/2zip/median_heuristic.py (deflated 56%)
updating: content/2zip/klcpd.py (deflated 72%)
updating: content/2zip/util.py (deflated 70%)
updating: content/2zip/mmd_util.py (deflated 60%)
updating: content/2zip/run_klcpd.py (deflated 62%)
updating: content/2zip/data_loader.py (deflated 69%)
  adding: content/2zip/experiment_log/ (stored 0%)
  adding: content/2zip/experiment_log/hasc.wnd-30.lambda_ae-0.001000.lambda_real-1.000000.clip-0.100000.trn.log (deflated 57%)
  adding: content/2zip/experiment_log/.ipynb_checkpoints/ (stored 0%)
  adding: content/2zip/experiment_log/hasc.wnd-30.lambda_ae-0.001000.lambda_real-1.000000.clip-0.100000/ (stored 0%)
  adding: content/2zip/experiment_log/usc.trn.log/ (stored 0%)
  adding: content/2zip/experiment_log/beedance-2.wnd-30.lambda_ae-0.001000.lambda_real-1.000000.clip-0.100000.trn.log (deflated 80%)
  adding: content/2zip/experiment_log/beedance-2.wn

#### Experiments with new architecture

In [12]:
###$ python run_klcpd.py --dataroot ./data --dataset beedance --wnd_dim_list 25 --max_iter 2000 --batch_size 64 
# dataset
parser = argparse.ArgumentParser(description='interface of running experiments for MMD baselines')
#parser.add_argument('--dataroot', type=str, default='beedance', required=True, help='[ ./data/simulation | ./data ] prefix path to data directory')
parser.add_argument('--dataroot', type=str, default='/content/MSD22/data', help = 'default version')
parser.add_argument('--dataset', type=str, default='hasc', help='dataset name ')
parser.add_argument('--gpu', type=int, default=0, help='gpu device id')

# hyperparameters for grid search
parser.add_argument('--wnd_dim_list', type=int, nargs='+', default=[5, 10, 15, 20, 25, 30], help='list of window dimensions')
parser.add_argument('--lambda_ae', type=float, default=1e-3, help='list of lambda_ae, coefficient of reconstruction loss')
parser.add_argument('--lambda_real', type=float, default=1, help='list of lambda_real, coefficient of MMD2(X_p_enc, X_f_enc)')
parser.add_argument('--max_iter', type=int, default=2000, help='max iteration for training')
parser.add_argument('--batch_size', type=int, default=16, help='batch_size for training')
parser.add_argument('--eval_freq', type=int, default=25, help='evaluation frequency per batch updates')
parser.add_argument('--weight_clip', type=float, default=.1, help='weight clipping for netD')

# experiment log
parser.add_argument('--save_dir', type=str, default='experiment_log', help='experiment directory for saving train log and models')

# sanity check
args, unknown = parser.parse_known_args()

dataroot = args.dataroot
dataset = args.dataset
gpu = args.gpu
wnd_dim_list = args.wnd_dim_list
lambda_ae = args.lambda_ae
lambda_real = args.lambda_real
max_iter = args.max_iter
batch_size = args.batch_size
eval_freq = args.eval_freq
weight_clip = args.weight_clip

In [11]:
!git clone https://github.com/BogChamp/MSD22.git

fatal: destination path 'MSD22' already exists and is not an empty directory.


In [15]:
import tensorflow as tf
from torch.utils import data


#### HASC



def ts_samples(mbatch, win):

    x = mbatch[:,1:win+1]
    y = mbatch[:,-win:]
    lbl = mbatch[:,0]
    return x, y, lbl

####


def load_hasc_ds(path, window, mode='train'):

    X, lbl = extract_windows(path, window, 5)

    if mode == "all":
        return X, lbl

    train_size = floor(0.8 * X.shape[0])
    print(train_size)
    if mode =="train":
        trainx = X[0:train_size]
        trainlbl =lbl[0:train_size]
        idx = np.arange(trainx.shape[0])
        np.random.shuffle(idx)
        trainx = trainx[idx,]
        trainlbl = trainlbl[idx]
        print('train samples : ', train_size)
        return trainx, trainlbl

    else:
        testx = X[train_size:]
        testlbl = lbl[train_size:]
        print('test shape {} and number of change points {} '.format(testx.shape, len(np.where(testlbl>0)[0])))

        return testx, testlbl



def extract_windows(path, window_size, step):

    dataset = sio.loadmat(path+"hasc.mat")
    window_size
    windows = []
    lbl = []
    first = True
    num_cp = 0
    x = np.array(dataset['Y'])
    #xs = ts[:,0]
    cp = np.array(dataset['L'])
    #cp = cp[:,0]

    ts = np.sqrt(np.power(x[:, 0], 2) + np.power(x[:, 1], 2) + np.power(x[:, 2], 2))
    for i in range(0, ts.shape[0] - window_size, step):
        windows.append(np.array(ts[i:i + window_size]))
        # print("TS",ts[i:i+window_size])
        is_cp = np.where(cp[i:i + window_size] == 1)[0]
        if is_cp.size == 0:
            is_cp = [0]
        else:
            num_cp += 1
        lbl.append(is_cp[0])

             # print(is_cp)

    print("number of samples : {} /  number of samples with change point : {}".format(len(windows), num_cp))
    windows = np.array(windows)

    return windows, np.array(lbl)

class Prepare_dataset(torch.utils.data.Dataset):    
    
    def __init__(self, path, labels):
        
        self.path = path
        self.labels = labels
    
    def __len__(self):
        
        return len(self.path)
    
    def __getitem__(self, index):
        series = self.path[index]
        tensor_of_image = torch.tensor(series)
        label = self.labels[index]
        
        return tensor_of_image, label 


    
def load_hasc_ds(path, window, mode='train'):

    X, lbl = extract_windows(path, window, 5)

    if mode == "all":
        return X, lbl

    train_size = floor(0.8 * X.shape[0])
    print(train_size)
    if mode =="train":
        trainx = X[0:train_size]
        trainlbl =lbl[0:train_size]
        idx = np.arange(trainx.shape[0])
        np.random.shuffle(idx)
        trainx = trainx[idx,]
        trainlbl = trainlbl[idx]
        print('train samples : ', train_size)
        return trainx, trainlbl

    else:
        testx = X[train_size:]
        testlbl = lbl[train_size:]
        print('test shape {} and number of change points {} '.format(testx.shape, len(np.where(testlbl>0)[0])))

        return testx, testlbl

def load_dataset(path, ds_name, win, bs, mode="train"):
    if ds_name == 'HASC':
        trainx, trainlbl = load_hasc_ds(path, window = 2 * win, mode=mode)
    elif ds_name == "USC":
        trainx, trainlbl = load_usc_ds(path, window = 2 * win, mode=mode)
    else:
        raise ValueError("Undefined Dataset.")

    trainlbl = trainlbl.reshape((trainlbl.shape[0], 1))
    print(trainx.shape, trainlbl.shape)
    
    dataset = np.concatenate((trainlbl, trainx), 1)

    print("dataset shape : ", dataset.shape)
    if mode == "train":
        data_train = Prepare_dataset(trainx, trainlbl)
        dataloader_train = torch.utils.data.DataLoader(data_train,
                                                       batch_size=bs,
                                                       shuffle=True)

    
        return dataloader_train
        
    
    
    if mode == "test":
        return dataset
    
    

In [16]:
path= "/content/MSD22/data/"
dataset = sio.loadmat(path+"hasc.mat")
dataset

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Mon Dec 18 11:22:20 2017',
 '__version__': '1.0',
 '__globals__': [],
 'Y': array([[0.34453952, 0.39477   , 0.36871999],
        [0.34232838, 0.39556832, 0.37001254],
        [0.34500567, 0.39362621, 0.36839557],
        ...,
        [0.48225509, 0.67834158, 0.09142372],
        [0.49178158, 0.67628665, 0.08959241],
        [0.48706243, 0.67298643, 0.09384147]]),
 'L': array([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]])}

In [17]:
import math

def extract_windows(path, window_size, step):

    dataset = sio.loadmat(path+"hasc.mat")
    windows = []
    lbl = []
    first = True
    num_cp = 0
    x = np.array(dataset['Y'])
    cp = np.array(dataset['L'])

    ts = np.sqrt(np.power(x[:, 0], 2) + np.power(x[:, 1], 2) + np.power(x[:, 2], 2))
    #print("ts", ts[:20] ,"\n")
    
    #print("ts.shape[0] - window_size", ts.shape[0] - window_size, "\n")
    
    for i in range(0, ts.shape[0] - window_size, step):
        windows.append(np.array(ts[i:i + window_size]))
        #print("TS",ts[i:i+window_size])
        is_cp = np.where(cp[i:i + window_size] == 1)[0]
        if is_cp.size == 0:
            is_cp = [0]
        else:
            #print(i)
            #print("is_cp", is_cp)
            num_cp += 1
        lbl.append(is_cp[0])

             # print(is_cp)

    print("number of samples : {} /  number of samples with change point : {}".format(len(windows), num_cp))
    windows = np.array(windows)

    return windows, np.array(lbl)

def load_hasc_ds(path, window, train_share = 0.8, step = 5, mode='train'):

    X, lbl = extract_windows(path, window, step)

    if mode == "all":
        return X, lbl

    train_size = math.floor(train_share * X.shape[0])
    print(train_size)
    if mode == "train":
        trainx = X[0:train_size]
        trainlbl =lbl[0:train_size]
        idx = np.arange(trainx.shape[0])
        np.random.shuffle(idx)
        trainx = trainx[idx,]
        trainlbl = trainlbl[idx]
        print('train samples : ', train_size)
        return trainx, trainlbl

    else:
        testx = X[train_size:]
        testlbl = lbl[train_size:]
        print('test shape {} and number of change points {} '.format(testx.shape, len(np.where(testlbl>0)[0])))

        return testx, testlbl
    
class Prepare_dataset(torch.utils.data.Dataset):    
    
    def __init__(self, path, labels):
        
        self.path = path
        self.labels = labels
    
    def __len__(self):
        
        return len(self.path)
    
    def __getitem__(self, index):
        series = self.path[index]
        tensor_of_image = torch.tensor(series)
        label = self.labels[index]
        
        return tensor_of_image, label 
        #return np.concatenate((label, tensor_of_image), 1)

def load_dataset(path, ds_name, win, bs, mode="train"):
    if ds_name == 'HASC':
        data, label = load_hasc_ds(path, window = 2 * win, mode=mode)
    elif ds_name == "USC":
        data, label = load_usc_ds(path, window = 2 * win, mode=mode)
    else:
        raise ValueError("Undefined Dataset.")

    label = label.reshape((label.shape[0], 1))
    print(data.shape, label.shape)
    
    #dataset = np.concatenate((trainlbl, trainx), 1)

    #print("dataset shape : ", dataset.shape)
    if mode == "train":
        data_train = Prepare_dataset(data, label)
        dataloader_train = torch.utils.data.DataLoader(data_train,
                                                       batch_size=bs,
                                                       shuffle=True)
        return dataloader_train
        
    
    
    if mode == "test":
        data_test = Prepare_dataset(data, label)
        dataloader_test = torch.utils.data.DataLoader(data_test,
                                                      batch_size=bs,
                                                      shuffle=False)

        return dataloader_test

In [18]:
# ========= Setup input argument =========#
parser = argparse.ArgumentParser(description='PyTorch Time series forecasting')
#parser.add_argument('--data_path', type=str, required=True, help='path to data in matlab format')
parser.add_argument('--data_path', type=str, default = '//content/klcpd_code/data/beedance/beedance-1.mat', help='path to data in matlab format')
parser.add_argument('--trn_ratio', type=float, default=0.6,help='how much data used for training')
parser.add_argument('--val_ratio', type=float, default=0.8,help='how much data used for validation')
parser.add_argument('--gpu', type=int, default=0, help='gpu device id')
parser.add_argument('--cuda', type=str, default=True, help='use gpu or not')
parser.add_argument('--random_seed', type=int, default=1126,help='random seed')
#parser.add_argument('--wnd_dim', type=int, required=True, default=10, help='window size (past and future)')
parser.add_argument('--wnd_dim', type=int, default=10, help='window size (past and future)')
parser.add_argument('--sub_dim', type=int, default=1, help='dimension of subspace embedding')

# RNN hyperparemters
parser.add_argument('--RNN_hid_dim', type=int, default=10, help='number of RNN hidden units')

# optimization
parser.add_argument('--batch_size', type=int, default=128, help='batch size for training')
parser.add_argument('--max_iter', type=int, default=100, help='max iteration for pretraining RNN')
parser.add_argument('--optim', type=str, default='adam', help='sgd|rmsprop|adam for optimization method')
parser.add_argument('--lr', type=float, default=3e-4, help='learning rate')
parser.add_argument('--weight_decay', type=float, default=0., help='weight decay (L2 regularization)')
parser.add_argument('--momentum', type=float, default=0.0, help='momentum for sgd')
parser.add_argument('--grad_clip', type=float, default=10.0, help='gradient clipping for RNN (both netG and netD)')
parser.add_argument('--eval_freq', type=int, default=50, help='evaluation frequency per generator update')

# GAN
parser.add_argument('--CRITIC_ITERS', type=int, default=5, help='number of updates for critic per generator')
parser.add_argument('--weight_clip', type=float, default=.1, help='weight clipping for crtic')
parser.add_argument('--lambda_ae', type=float, default=0.001, help='coefficient for the reconstruction loss')
parser.add_argument('--lambda_real', type=float, default=0.1, help='coefficient for the real MMD2 loss')

# save models  /content/exp_simulate/jumpingmean/save_RNN
parser.add_argument('--save_path', type=str,  default='/content/exp_simulate/jumpingmean/save_RNN',help='path to save the final model')

args, unknown = parser.parse_known_args()

In [19]:
windows, labels = extract_windows(path = path, window_size=60, step = 60)

number of samples : 656 /  number of samples with change point : 63


In [22]:
train_dataloader = load_dataset(path = path,
                                ds_name = "HASC", 
                                win = 60,
                                bs = 64,
                                mode="train")

number of samples : 7856 /  number of samples with change point : 1390
6284
train samples :  6284
(6284, 120) (6284, 1)


In [23]:
trainx, trainlbl = load_hasc_ds(path = path, window = 60*2, train_share = 0.8, step = 5, mode='train')
testx, testlbl = load_hasc_ds(path = path, window = 60*2, train_share = 0.8, step = 5, mode='test')

number of samples : 7856 /  number of samples with change point : 1390
6284
train samples :  6284
number of samples : 7856 /  number of samples with change point : 1390
6284
test shape (1572, 120) and number of change points 263 


In [26]:
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from  kl_cpd import *

In [57]:
args_dict = vars(args)
args_dict

{'data_path': '//content/klcpd_code/data/beedance/beedance-1.mat',
 'trn_ratio': 0.6,
 'val_ratio': 0.8,
 'gpu': 0,
 'cuda': True,
 'random_seed': 1126,
 'wnd_dim': 10,
 'sub_dim': 1,
 'RNN_hid_dim': 10,
 'batch_size': 128,
 'max_iter': 100,
 'optim': 'adam',
 'lr': 0.0003,
 'weight_decay': 0.0,
 'momentum': 0.0,
 'grad_clip': 10.0,
 'eval_freq': 50,
 'CRITIC_ITERS': 5,
 'weight_clip': 0.1,
 'lambda_ae': 0.001,
 'lambda_real': 0.1,
 'save_path': '/content/exp_simulate/jumpingmean/save_RNN',
 'D': 3,
 'var_dim': 3,
 'sqdist': 0.00415844153127809,
 'window': 10}

In [58]:
Y = dataset['Y']                                   # Y: time series data, time length x number of variables
L = dataset['L']                                   # L: label of anomaly, time length x 1
T, D = Y.shape
args_dict['sub_dim'] = args.sub_dim 
args_dict['D']= D 
args_dict['var_dim'] = args_dict['D'] * args_dict['sub_dim']


In [59]:
from sklearn.metrics.pairwise import euclidean_distances

sub_dim = args_dict['sub_dim']
Y_subspace = np.zeros((T, D, sub_dim))
for t in range(sub_dim, T):
    for d in range(D):
        Y_subspace[t, d, :] = Y[t-sub_dim+1:t+1, d].flatten()

        # Y_subspace is now T x (Dxsub_dim)
Y_subspace = Y_subspace.reshape(T, -1)


X = Y_subspace
max_n = min(10000, X.shape[0])
X[:max_n].shape
D2 = euclidean_distances(X[:max_n], squared=True)

args_dict['sqdist'] = np.median(D2[np.triu_indices_from(D2, k=1)])

In [64]:
args_dict

{'data_path': '//content/klcpd_code/data/beedance/beedance-1.mat',
 'trn_ratio': 0.6,
 'val_ratio': 0.8,
 'gpu': 0,
 'cuda': True,
 'random_seed': 1126,
 'wnd_dim': 10,
 'sub_dim': 1,
 'RNN_hid_dim': 10,
 'batch_size': 128,
 'max_iter': 100,
 'optim': 'adam',
 'lr': 0.0003,
 'weight_decay': 0.0,
 'momentum': 0.0,
 'grad_clip': 10.0,
 'eval_freq': 50,
 'CRITIC_ITERS': 5,
 'weight_clip': 0.1,
 'lambda_ae': 0.001,
 'lambda_real': 0.1,
 'save_path': '/content/exp_simulate/jumpingmean/save_RNN',
 'D': 3,
 'var_dim': 3,
 'sqdist': 0.00415844153127809,
 'window': 10}

In [39]:
class NetG(nn.Module):
    def __init__(self, args):
        super(NetG, self).__init__()
        #self.wnd_dim = args.wnd_dim
        self.wnd_dim = args['wnd_dim']
        self.var_dim = args['var_dim']
        self.D = args['D']
        #self.RNN_hid_dim = args.RNN_hid_dim
        self.RNN_hid_dim = args['RNN_hid_dim']

        self.rnn_enc_layer = nn.GRU(self.var_dim, self.RNN_hid_dim, num_layers=1, batch_first=True)
        self.rnn_dec_layer = nn.GRU(self.var_dim, self.RNN_hid_dim, num_layers=1, batch_first=True)
        self.fc_layer = nn.Linear(self.RNN_hid_dim, self.var_dim)

    # X_p:   batch_size x wnd_dim x var_dim (Encoder input)
    # X_f:   batch_size x wnd_dim x var_dim (Decoder input)
    # h_t:   1 x batch_size x RNN_hid_dim
    # noise: 1 x batch_size x RNN_hid_dim
    def forward(self, X_p, X_f, noise):
        X_p_enc, h_t = self.rnn_enc_layer(X_p)
        X_f_shft = self.shft_right_one(X_f)
        hidden = h_t + noise
        Y_f, _ = self.rnn_dec_layer(X_f_shft, hidden)
        output = self.fc_layer(Y_f)
        return output


    def shft_right_one(self, X):
        X_shft = X.clone()
        X_shft[:, 0, :].data.fill_(0)
        X_shft[:, 1:, :] = X[:, :-1, :]
        return X_shft

class NetD(nn.Module):
    def __init__(self, args):
        super(NetD, self).__init__()

        #self.wnd_dim = args.wnd_dim
        self.wnd_dim = args['wnd_dim']
        #self.var_dim = data.var_dim
        self.var_dim = args['var_dim']
        self.D = args['D']
        #self.RNN_hid_dim = args.RNN_hid_dim
        self.RNN_hid_dim = args['RNN_hid_dim']

        self.rnn_enc_layer = nn.GRU(self.var_dim, self.RNN_hid_dim, batch_first=True)
        self.rnn_dec_layer = nn.GRU(self.RNN_hid_dim, self.var_dim, batch_first=True)

    def forward(self, X):
        X_enc, _ = self.rnn_enc_layer(X)
        X_dec, _ = self.rnn_dec_layer(X_enc)
        return X_enc, X_dec



In [41]:
netG = NetG(args_dict)
netD = NetD(args_dict)

In [65]:
args.wnd_dim

10

In [44]:
args_dict['window'] = args.wnd_dim

In [46]:


model =  KLCPD(
      netG = netG,
      netD = netG,
      args = args_dict,
      train_dataset = trainx,
      test_dataset = testx,
      num_workers = 2
    )

train_loader = model.train_dataloader()
val_loader = model.val_dataloader()

trainer = Trainer(accelerator='gpu', devices=1)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [51]:
from tqdm import tqdm

In [ ]:
epochs = 5


model =  KLCPD(
      netG = netG,
      netD = netG,
      args = args_dict,
      train_dataset = trainx,
      test_dataset = testx,
      num_workers = 2
    )

optimizer = model.configure_optimizers()

train_loader = model.train_dataloader()
val_loader = model.val_dataloader()

train_losses = []
val_losses = []

total_iterations = 0
for epoch in tqdm(range(epochs)):
    
    iteration = 0
    train_losses_4_iters = []
    
    for index, batch in enumerate(train_loader):
        
        if iteration != 5:
            optimizer_idx = iteration % 2
            loss = model.training_step(batch, index, optimizer_idx)
            train_losses_4_iters.append(float(loss))
            print("train_losses_4_iters", train_losses_4_iters)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            iteration += 1
            
            print("iteration", iteration)
        
        else:
            model.eval()
            val_loss = model.validation_step(batch, index)
            print("val_loss", val_loss)
            val_losses.append(float(val_loss.detach()))
            print("val_losses", val_losses)
            train_losses.append(np.mean(train_losses_4_iters))
            print("train_losses", train_losses)
            
            val_loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            iteration = 0
            train_losses_4_iters = []
            
            clear_output(wait=True)
            plt.figure(figsize=(18, 12))
            
            plt.plot(val_losses, label = "validation loss", color = "blue")
            plt.plot(train_losses, label = "training loss", color = "red")
            plt.yscale("log")
            plt.ylabel("log Info NCE loss", fontsize = 14)
            plt.xlabel("iterations", fontsize = 14)
            plt.legend(fontsize = 14)
            plt.show();
            model.train()